## Scrape the latest 2000 submissions from any subreddit. I've scraped r/SomebodyMakeThis

In [ ]:
!pip install praw
import praw
reddit=praw.Reddit(client_id='yourClientID',client_secret='yourClientSecret',user_agent='Python bot v1.0');
subreddit="SomebodyMakeThis"


In [ ]:
import requests

url = "https://api.pushshift.io/reddit/search/submission"

def crawl_page(subreddit: str, last_page = None):
  """Crawl a page of results from a given subreddit.

  :param subreddit: The subreddit to crawl.
  :param last_page: The last downloaded page.

  :return: A page or results.
  """
  params = {"subreddit": subreddit, "size": 500, "sort": "desc", "sort_type": "created_utc"}
  if last_page is not None:
    if len(last_page) > 0:
      # resume from where we left at the last page
      params["before"] = last_page[-1]["created_utc"]
    else:
      # the last page was empty, we are past the last page
      return []
  results = requests.get(url, params)
  if not results.ok:
    # something wrong happened
    raise Exception("Server returned status code {}".format(results.status_code))
  return results.json()["data"]

In [ ]:
import time

def crawl_subreddit(subreddit, max_submissions = 2000):
  """
  Crawl submissions from a subreddit.

  :param subreddit: The subreddit to crawl.
  :param max_submissions: The maximum number of submissions to download.

  :return: A list of submissions.
  """
  submissions = []
  last_page = None
  while last_page != [] and len(submissions) < max_submissions:
    last_page = crawl_page(subreddit, last_page)
    submissions += last_page
    time.sleep(3)
  return submissions[:max_submissions]

In [ ]:
lastest_submissions = crawl_subreddit(subreddit)

## Getting author, idea, description, upvote_ratio from lastest_submissions

In [ ]:
title=[]
author=[]
description=[]
url=[]
for subs in lastest_submissions:
  title.append(subs["title"])
  author.append(subs["author"])
  try:
    description.append(subs["selftext"])
  except KeyError:
    description.append("")
  url.append(subs["url"])


In [ ]:
import pandas as pd
df=pd.DataFrame()
df["title"]=title
df["author"]=author
df["description"]=description
df["url"]=url

In [ ]:
df.index.rename('index',inplace=True)

In [ ]:
df.to_csv("submissions.csv") # Storing the data as a csv file

## Storing the data in Firebase

In [ ]:
#get your key from firebase console
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore
cred = credentials.Certificate("yourKeyFileName.json")
db = firestore.client()
tmp = df.to_dict(orient='records')
for i in range(0,2000):
  db.collection(u'submissions').document(i).set(tmp[i])
  print(i)